In [1]:
%pip install ollama pandas numpy
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import json
import numpy as np
from ollama import Client

data = pd.read_csv("../task2/failed_extracts.csv")
data


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


,Unnamed: 0,ingredient,amount,ingr_annotation,amount_annotation,norm_value,norm_unit,nutrition
0,5,passierte tomaten,300 g,"{""gewicht"": 300, ""einheit"": ""g""}","{""zutat"": ""Tomate"", ""eigenschaft"": ""passiert""}",300.000,g,NaN
1,15,bio hähnchenschlegel,4 stk.,"{""anzahl"": 4, ""einheit"": ""stk.""}","{""zutat"": ""H\u00e4hnchenschlegel"", ""eigenschaf...",4.000,stk,NaN
2,20,snackkarotten,80 g,"{""gewicht"": 80, ""einheit"": ""g""}","{""zutat"": ""Karotte"", ""eigenschaft"": ""Snack-""}",80.000,g,NaN
3,22,weißwein trocken,100 ml,"{""volumen"": 100, ""einheit"": ""ml""}","{""zutat"": ""Wein"", ""eigenschaft"": ""trocken""}",0.100,l,NaN
4,31,schokoriegel,1 stk.,"{""anzahl"": 1, ""einheit"": ""stk.""}","{""zutat"": ""Schokoriegel""}",1.000,stk,NaN
...,...,...,...,...,...,...,...,...
109,891,küchentuch,1 stk.,"{""anzahl"": 1, ""einheit"": ""stk.""}","{""zutat"": ""K\u00fcchentuch"", ""eigenschaft"": ""S...",1.000,stk,NaN
110,896,sellerieknolle gegart,50 g,"{""gewicht"": 50, ""einheit"": ""g""}","{""zutat"": ""Sellerieknolle"", ""eigenschaft"": ""ge...",50.000,g,NaN
111,898,spitzpaprika rot,30 g,"{""gewicht"": 30, ""einheit"": ""g""}","{""zutat"": ""Spitzpaprika"", ""eigenschaft"": ""rot""}",30.000,g,NaN
112,902,norderneyer seeluftschinken,6 stück,"{""anzahl"": 6, ""einheit"": ""st\u00fcck""}","{""zutat"": ""Schinken"", ""eigenschaft"": ""seeluft""}",6.000,stk,NaN


In [ ]:
llama_3_3 = Client(host="http://intern.schlaubox.de:11434", timeout=500)

def extract_ingredients_json(ingr_txt):
    messages = [{ "role" : "user", "content" :
    f"""Your task is to identify cooking ingredients in texts and output them in the nominative singular.
If characteristics such as color, condition, processing, quality, or origin are mentioned for an ingredient, output these characteristics as well.
Generate a json object for the output.
Always include a "ingredient" as the key for the ingredient name. Also add a "properties" array for all previously mentioned properties. If none were found, do not use this key.
Make sure all values are in the German basic form, like in the following example.

An example for this is:
    input: "garnelen frisch groß"
    output: ```json
        {{
            "ingredient": "Garnele",
            "properties": ["frisch", "groß"]
        }}
    ```

Now identify the ingredients in this text: {ingr_txt}"""}]
    llm_analysis = llama_3_3.chat(model='llama3.3:70b', messages=messages)
    return llm_analysis['message']['content']

data['ingr_json'] = data.apply(lambda row: extract_ingredients_json(row['ingredient']), axis=1)
data

ConnectTimeout: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

In [ ]:

def extract_quantities_json(amount_txt):
    messages = [{ "role" : "user", "content" :
    f"""Your task is to identify quantities, weights, and volumes in cooking ingredients in texts.
Generate a json object for the output.
Always use 'amount' as the key for quantities, 'weight' for weights, and 'volume' for volumes.
Always specify a 'unit' key for the unit found, e.g., l, g, Stück, Prise, msp, Tafel, Riegel or Esslöffel!
Do not fabricate or include any other information in the output!
Make sure all values are in the German basic form, like in the following example.

A few examples for this is:
Example 1:
    input: "1 Teelöffel Salz"
    output: ```json
        {{
            "amount": 1,
            "unit": "teelöffel"
        }}
    ```

Example 2:
    input: "2 Müsliriegel"
    output: ```json
        {{
            "amount": "2",
            "unit": "Riegel"
        }}
    ```

Example 3:
    input: "2 Tafeln Schokolade"
    output: ```json
        {{
            "amount": "2",
            "unit": "Tafel"
        }}

Example 4:
    input: "3 Kleckse Butter"
    output: ```json
        {{
            "amount": "3",
            "unit": "Klecks"
        }}


Example 5:
    input: "1 Schuss Rapsöl"
    output: ```json
        {{
            "amount": "1",
            "unit": "Schuss"
        }}

Example 6:
    input: "3 Päckchen frische Heidelbeeren"
    output: ```json
        {{
            "amount": "3",
            "unit": "Pack"
        }}

Example 7:
    input: "1/2 Frucht Orangensaft frisch gepresst"
    output: ```json
        {{
            "amount": "0.5",
            "unit": "Stück"
        }}
    ```

Example 8:
    input: "1/2 Kopf frischer Salat"
    output: ```json
        {{
            "amount": "0.5",
            "unit": "Kopf"
        }}
    ```

Example 9:
    input: "1 Spritzer Wein"
    output: ```json
        {{
            "amount": "1",
            "unit": "Spritzer"
        }}
    ```

Now identify quantities, weights, and volumes in this text: {amount_txt}"""}]
# Maybe add: spritzer, blatt, kopf, stk.

    llm_analysis = llama_3_3.chat(model='llama3.3:70b', messages=messages)
    return llm_analysis['message']['content']

data['amount_json'] = data.apply(lambda row: extract_ingredients_json(row['amount']), axis=1)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0.1       910 non-null    int64  
 1   Unnamed: 0         910 non-null    int64  
 2   ingredient         910 non-null    object 
 3   amount             910 non-null    object 
 4   ingr_annotation    910 non-null    object 
 5   amount_annotation  910 non-null    object 
 6   norm_value         910 non-null    float64
 7   norm_unit          910 non-null    object 
 8   nutrition          601 non-null    object 
dtypes: float64(1), int64(2), object(6)
memory usage: 64.1+ KB
